# 0. Mess around space

In [1]:
import csv
from impt_functions import *

In [2]:
dt_dict_overall = {'Run1': [0.8, 0.9], 'Run2': [3.0, 2], 'Run3': [10, 11]}
save_dict_2_csv(dt_dict_overall, 'dt_overall_results.csv')

('Run1', [0.8, 0.9])
['Run1']
[0.8, 0.9]
('Run2', [3.0, 2])
['Run2']
[3.0, 2]
('Run3', [10, 11])
['Run3']
[10, 11]


# 1. Imports

In [3]:
from impt_functions import *
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import pandas as pd 
import numpy as np
from fairlearn.reductions import ExponentiatedGradient, GridSearch, DemographicParity, EqualizedOdds, \
    TruePositiveRateParity, FalsePositiveRateParity, ErrorRateParity, BoundedGroupLoss
from fairlearn.metrics import *
from raiwidgets import FairnessDashboard
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
%matplotlib inline

# 2. Prepare data

In [4]:
data = get_data('/home/mackenzie/git_repositories/delayedimpact/data/simData_oom100.csv')

       score  repay_probability  race  repay_indices
0        601              75.21     1              1
1        693              95.15     1              1
2        791              98.62     1              1
3        637              86.69     1              1
4        775              98.45     1              1
...      ...                ...   ...            ...
99995    797              98.73     1              1
99996    562              58.57     1              1
99997    687              94.60     1              1
99998    589              70.61     1              1
99999    555              52.97     1              0

[100000 rows x 4 columns]


In [5]:
X_train, X_test, y_train, y_test, race_train, race_test, sample_weight_train, sample_weight_test = prep_data(data=data, test_size=0.3, weight_index=1)

Here are the x values:  [[601   1]
 [693   1]
 [791   1]
 ...
 [687   1]
 [589   1]
 [555   1]] 

Here are the y values:  [1 1 1 ... 1 1 0]
Sample weights are all equal.


# Decision Tree Classifier

## Train DTC classifier + Collect Predictions
NOTE: atm sample_weight are all 1s

In [6]:
# Reference: https://www.datacamp.com/community/tutorials/decision-tree-classification-python

# Initialize classifier:
clf = DecisionTreeClassifier()

# Train the classifier:
model = clf.fit(X_train,y_train)

# Make predictions with the classifier:
y_predict = model.predict(X_test)

# Scores on test set
test_scores = model.predict_proba(X_test)[:, 1]

### Evaluation of classifier overall

In [7]:
# Metrics
models_dict = {"Unmitigated": (y_predict, test_scores)}
get_metrics_df(models_dict, y_test, race_test)

,Unmitigated
Overall selection rate,0.729567
Demographic parity difference,0.450625
Demographic parity ratio,0.424364
------,
Overall balanced error rate,0.151732
Balanced error rate difference,0.0170221
------,
True positive rate difference,0.190934
True negative rate difference,0.156889
False positive rate difference,0.156889


In [11]:
cm = confusion_matrix(y_test, y_predict)
print(cm)
results_dict = classification_report(y_test, y_predict, output_dict=True)
print(classification_report(y_test, y_predict))
evaluation_outcome_rates(y_test, y_predict, sample_weight_test)
get_f1_scores(y_test, y_predict)
get_selection_rates(y_test, y_predict, race_test, 0)

[[ 6644  2036]
 [ 1469 19851]]
              precision    recall  f1-score   support

           0       0.82      0.77      0.79      8680
           1       0.91      0.93      0.92     21320

    accuracy                           0.88     30000
   macro avg       0.86      0.85      0.86     30000
weighted avg       0.88      0.88      0.88     30000

TNR=TN/(TN+FP)=  0.7654377880184332
TPR=TP/(FP+FN)=  0.9310975609756098
FNER=FN/(FN+TP)=  0.06890243902439025
FPER=FP/(FP+TN)=  0.23456221198156682
F1 score micro: 
0.8831666666666667
F1 score weighted: 
0.8819608768731572
F1 score binary: 
0.9188788853657973
Selection Rate Overall:  0.7295666666666667


The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


In [9]:
print(results_dict)

{'0': {'precision': 0.8189325773450018, 'recall': 0.7654377880184332, 'f1-score': 0.7912820818198059, 'support': 8680}, '1': {'precision': 0.9069767441860465, 'recall': 0.9310975609756098, 'f1-score': 0.9188788853657973, 'support': 21320}, 'accuracy': 0.8831666666666667, 'macro avg': {'precision': 0.8629546607655241, 'recall': 0.8482676744970215, 'f1-score': 0.8550804835928016, 'support': 30000}, 'weighted avg': {'precision': 0.881502631913371, 'recall': 0.8831666666666667, 'f1-score': 0.8819608768731572, 'support': 30000}}


### Evaluation of classifier by race

In [ ]:
evaluation_by_race(X_test, y_test, race_test, y_predict, sample_weight_test)

### Delayed impact calculated by race

In [ ]:
calculate_delayed_impact(X_test, y_test, y_predict, race_test)

### Fairness Metric Evaluation of classifier

In [ ]:
print_fairness_metrics(y_test, y_predict, race_test)

## Exponentiated Gradient Reduction Alg for Adding Fairness Constraints

### Demographic Parity

In [ ]:
eg_dp = add_contraint(model, 'DP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### Equalized Odds

In [ ]:
eg_eo = add_contraint(model, 'EO', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### EOO (True Positive Rate Parity)

In [ ]:
eg_tprp = add_contraint(model, 'TPRP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### False Positive Rate Parity

In [ ]:
eg_fprp = add_contraint(model, 'FPRP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### Error Rate Parity

In [ ]:
eg_erp = add_contraint(model, 'ERP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### Bounded Group Loss (TODO: issue, need to figure out loss parameter)

In [ ]:
#add_contraint(model, 'BGL', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

## Grid Search Reduction Alg for Adding Fairness Constraints

### Demographic Parity

In [ ]:
gs_dp = add_contraint(model, 'DP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_dp.lambda_vecs_
print(lambda_vecs[0])

The next few cells come from: https://github.com/fairlearn/fairlearn/blob/main/notebooks/Binary%20Classification%20with%20the%20UCI%20Credit-card%20Default%20Dataset.ipynb

Note: we train multiple models corresponding to different trade-off points between the performance metric (balanced accuracy) and fairness metric.

In [ ]:
grid_search_show(gs_dp, demographic_parity_difference, y_predict, X_test, y_test, race_test, 'DemParityDifference','GS DPD', models_dict, 0.3)

In [ ]:
models_dict.pop('GS DPD')
models_dict

### Equalized Odds Used

In [ ]:
gs_eo = add_contraint(model, 'EO', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_eo.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
grid_search_show(gs_eo, equalized_odds_difference, y_predict, X_test, y_test, race_test, 'EOddsDifference','GS EO', models_dict, 0.3)

In [ ]:
models_dict.pop('GS EO')
models_dict

### EOO (True Positive Rate Parity)

In [ ]:
gs_tprp = add_contraint(model, 'TPRP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_tprp.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
grid_search_show(gs_tprp, true_positive_rate_difference, y_predict, X_test, y_test, race_test, 'TPRPDifference','GS TPRP', models_dict, 0.3)

In [ ]:
models_dict.pop('GS TPRP')
models_dict

### False Positive Rate Parity

In [ ]:
gs_fprp = add_contraint(model, 'FPRP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_fprp.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
# NOTE: the below models are the same!!

In [ ]:
grid_search_show(gs_fprp, false_positive_rate_difference, y_predict, X_test, y_test, race_test, 'FPRPDifference','GS FPRP', models_dict, 0.4)

In [ ]:
models_dict.pop('GS FPRP')
models_dict

### Error Rate Parity

In [ ]:
gs_erp = add_contraint(model, 'ERP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_erp.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
# Fairlearn doesnt have an erp difference metric for the below
#grid_search_show(gs_erp, error_difference, y_predict, X_test, y_test, race_test, 'ERDifference','GS ERP', models_dict, 0.3)
#models_dict.pop('GS FPRP')
#models_dict

### Bounded Group Loss (issue, need to figure out loss parameter)

In [ ]:
#gs_bgl = add_contraint(model, 'BGL', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
#lambda_vecs = gs_dp.lambda_vecs_
#print(lambda_vecs[0])